# A Note on this Notebook
This can be run as either a python/ipython script or as a notebook.
It generates a firedrake `DumbCheckpoint` file called `true-fields.h5` containing the values of $u_{true}$ and $q_{true}$ in `Function`s named `u_true` and `q_true` respectively.
The investigation continues in another notebook which uses these fields.

# Problem Description

We want to find out how the solution of our inverse problem converges as we increase the number of points for both the new and traditional methods of data interpolation.

If we have what is known as **"posterior consistency"** then we expect that the error in our solution, when compared to the true solution, will always decrease as we increase the number of points we are assimilating.

## Posterior Consistency

From a Bayesian point of view, the regularisation we choose and the weighting we give it encode information about our assumed prior probability distribution of $q$ before we start assimilating data (adding observations).
Take, for example, the regularisation used in the this problem

$$
\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx
$$

which asserts a prior that the solution $q$ which minimises $J$ should be smooth and gives a weighting $\alpha$ to the assertion.
If we have posterior consistency, the contribution of increasing numbers of measurements $u_{obs}$ should increase the weighting of our data relative to our prior and we should converge towards the true solution.

## Hypothesis

Our two methods minimise two different functionals. 
The first minimises $J$

$$J[u, q] = 
\underbrace{\frac{1}{2}\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \text{P0DG}(\Omega_v))}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

whilst the second minimises $J''$

$$J''[u, q] = 
\underbrace{\frac{1}{2}\int_{\Omega}\left(\frac{u_{interpolated} - u}{\hat{\sigma}}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}}.$$

**where $\hat{\sigma}$ is an appropriate value for the given regularisation term $\alpha$ found with an l-curve analysis.**

As set up here increasing the number of points to assimilate has the effect of increasing the size of the misfit term in $J$ (with a weight proportional to each measurement's variance $\sigma$) so we expect to converge to $q_{true}$ as the number of measurements increases.

As we increase the number of measurements in $J''$ we hope that our calculated $u_{interpolated}$ approaches $u$ (to therefore minimise the misfit). There is, however, no mechanism to cause the misfit term to increase relative to the regularizatuion term.

We therefore predict that minimising $J$ will display posterior consistency and that minimising the various $J'$ for each $u_{interpolated}$ will not.

## Hypothesis Amendment! A note on finite element method error
Note that our solutions all exist in finite element spaces which are usually approximations of a true solution with some error that (hopefully) decreases as mesh density increase and solution space order increase.
Since I am comparing to a solution $u_true$ in CG2 space I expect, at best, that we will converge to $u_true$ when we have, on average, enough points per cell to fully specify the lagrange polynomials in that cell.
Were we in CG1 this would be 3 points per cell (I can't remember how many we would need for CG2!) to give convergence if those measurements had no noise.
Since our measurements are noisy I do not expect actual convergence, but I anticipate some slowing in convergence.

# Setup

In [ ]:
from scipy.interpolate import (
    LinearNDInterpolator,
    NearestNDInterpolator,
    CloughTocher2DInterpolator,
    Rbf,
)

import matplotlib.pyplot as plt
import firedrake
import firedrake_adjoint

from firedrake import Constant, cos, sin

import numpy as np
from numpy import pi as π
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os

currentdir = os.path.dirname(os.path.realpath('__file__'))

In [ ]:
mesh = firedrake.UnitSquareMesh(32, 32)

# Solution Space
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)

# q (Control) Space
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

## Fake $q_{true}$

In [ ]:
seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q, name='q_true')
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z

        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)

        q_true += mode

print('Made fake q_true')

## Fake $u_{true}$

In [ ]:
from firedrake import exp, inner, grad, dx
u_true = firedrake.Function(V, name='u_true')
v = firedrake.TestFunction(V)
f = Constant(1.0)
k0 = Constant(0.5)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q_true) * inner(grad(u_true), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u_true, bc)

print('Made fake u_true')

In [ ]:
# Clear tape since don't need to have taped above
tape = firedrake_adjoint.get_working_tape()
tape.clear_tape()

## Data Output
We save our fields to a firedrake checkpoint file.

In [ ]:
filename = os.path.join(currentdir, 'true-fields')

with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_CREATE) as chk:
    chk.store(q_true)
    chk.store(u_true)

Make sure they have saved...

In [ ]:
with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_READ) as chk:
    chk.load(q_true, name='q_true')
    chk.load(u_true, name='u_true')

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u_true, axes=axes, shading='gouraud')
fig.colorbar(colors);